In [1]:
import psycopg2
import configparser

## Connect and verify Data Model

### Set up connection to Redshift Cluster

In [4]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
aws_db_paremeters = ("""
host = {}
dbname = {}
user = {}
password = {}
port = {}
""").format(*config['CLUSTER'].values())

conn = psycopg2.connect(aws_db_paremeters)
cur = conn.cursor()

### Staging tables rows of data

In [14]:
cur = conn.cursor()
try:
    # cur.execute("SELECT COUNT(*) FROM staging_flights")
    cur.execute("SELECT COUNT(*) FROM fact_flights")
    conn.commit()
except Exception as e:
    print(e)
row = cur.fetchone()
print(row)

(1232410,)


## Execute analysis

#### "For Southwest Airline, how many landings does their 737 make in 2021?"

In [15]:
WN_737_landing = """
SELECT SUM(ff.departures_performed) AS flights 
FROM fact_flights ff
INNER JOIN dim_aircraft_code dac ON ff.aircraft_type = dac.ac_typeid
WHERE ff.unique_carrier_entity = '06725'
AND dac.short_name = 'B737-7'
AND ff.year = 2021
"""
conn.close()
conn = psycopg2.connect(aws_db_paremeters)
cur = conn.cursor()
try:
    cur.execute(WN_737_landing)

except Exception as e:
    print(e)
row = cur.fetchone()
print(row)

(Decimal('639945.00'),)


#### "For Airbus A321, what are the average route lengh chosen by US airlines?"

In [17]:
A321_avg_route_length = """
SELECT AVG(ff.distance) AS avg_distance 
FROM fact_flights ff
INNER JOIN dim_aircraft_code dac ON ff.aircraft_type = dac.ac_typeid
WHERE dac.short_name LIKE '%A321%'
AND ff.distance > 100
"""
conn.close()
conn = psycopg2.connect(aws_db_paremeters)
cur = conn.cursor()
try:
    cur.execute(A321_avg_route_length)

except Exception as e:
    print(e)
row = cur.fetchone()
print(row)

(Decimal('1203.92'),)


#### "What are the most popular destinations for helicotpers?"

In [21]:
Helicopter_dest = """
SELECT ff.DEST, dapc.city_name, SUM(ff.departures_performed) AS arrivals
FROM fact_flights ff
JOIN dim_aircraft_code dac ON ff.aircraft_type = dac.ac_typeid
JOIN dim_aircraft_group dag ON dac.ac_group = dag.ac_group
JOIN  dim_airport_code dapc ON ff.dest = dapc.airport_code
WHERE dag.ac_group_description = 'Helicopter/Stol'
GROUP BY ff.DEST, dapc.city_name
ORDER BY arrivals DESC
LIMIT 5;
"""

conn = psycopg2.connect(aws_db_paremeters)
cur = conn.cursor()
try:
    cur.execute(Helicopter_dest)

except Exception as e:
    print(e)
result_set = cur.fetchall()

for row in result_set:
    print(row)

conn.close()

('OME', 'Nome, AK', Decimal('2508.00'))
('KQA', 'Akutan, AK', Decimal('2409.00'))
('7AK', 'Akun, AK', Decimal('2381.00'))
('1G4', 'Peach Springs, AZ', Decimal('1100.00'))
('SCC', 'Deadhorse, AK', Decimal('873.00'))
